# 🧠 Aide pour le Challenge CTF - Data Leakage
Dans ce challenge, tu vas entraîner un modèle de machine learning sur un jeu de données de startups.
Le but est de prédire si une startup va réussir (`Success = 1`) ou échouer (`Success = 0`)


## 🕵️ Étapes conseillées

**Explorer les données** : affiche quelques lignes


**Vérifier la corrélation** entre les colonnes


Est-ce qu’une information est trop prédictive, comme un identifiant ou un code qui encode le succès ?

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv('D:\Documents 2\VScode projects\ROBAI2\challenges\data\startup.csv')

In [21]:
print(df.head())
print(df.info())
print(df.describe())
df['Code_Prediction'] = df['Startup Code'].str.contains('SUCCESS').astype(int)
accuracy = (df['Success'] == df['Code_Prediction']).mean()
print(f"Leakage-based accuracy: {accuracy:.2%}")

   Founding Year  Team Size  Funding Amount (USD)  Number of AI Patents  \
0           2016         58            2757901.08                     2   
1           2013          7            4904270.76                     3   
2           2022         29            3789599.98                     5   
3           2020         29            4007445.52                     1   
4           2017         45            8457710.09                     5   

   Website Visits per Month CEO Name    Startup Code  Success  
0                      3256    Heidi     S-FAIL-2824        0  
1                      7358      Eve     S-FAIL-1409        0  
2                      7355    Alice  S-SUCCESS-5506        1  
3                      4538      Eve     S-FAIL-5012        0  
4                      5394     Dana  S-SUCCESS-4657        1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  -

In [23]:
# Ensure only numeric columns are used for correlation
numeric_df = df.select_dtypes(include=[np.number])
correlation_matrix = numeric_df.corr()
print(correlation_matrix['Success'].sort_values(ascending=False))

Success                     1.000000
Code_Prediction             1.000000
Funding Amount (USD)        0.710880
Team Size                   0.328955
Number of AI Patents        0.193958
Website Visits per Month    0.149127
Founding Year              -0.080138
Name: Success, dtype: float64


## 🧩 Objectif final

Supprime les colonnes qui biaisent l’apprentissage.

Tu dois entraîner un modèle **qui généralise bien**